<a href="https://colab.research.google.com/github/BrendanL72/Hospitalization-Risk/blob/main/arimamodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
dfadl = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/MobileEdgeTaskCodeVisitSummaryJanuary-AugustSortedAfterDeIdentify.xlsx')
dfadl.nunique()

DeIdentify ID               210
ProviderID                    3
PayerGroup                    6
Caregiver ID                191
VisitDuration               307
ServiceTypeName              15
VisitDate                   243
ActualTimeIn                412
ActualTimeOut               390
txtBreakTimesValue            0
VisitDate2                  243
ScheduledTimeIn             148
ScheduledTimeOut            148
TaskCodeGroupName            60
HasBeenDone                   2
Reason                        7
Comments                   2490
NoSignature                   2
RectangleVoiceCaptured        1
SignatureDate             22726
dtype: int64

In [11]:
dfcrf = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/ClientReviewFormRemovingData.xlsx')
dfcrf.nunique()

Time                                               369
Masked Client ID                                   167
Date of Review                                     144
COVID -19                                            2
Which Review is this                                 3
                                                  ... 
Details of Schedule changes                         20
Are you happy with the services being provided?      2
If No make notes here                                1
Any Questions or Concerns?                          98
Case Manager Notes                                 243
Length: 67, dtype: int64

In [12]:
# clientarr = dfcrf['Masked Client ID'].unique()
# devicearr = df['device_id'].unique()
# #assign empty client column to sensor dataframe
# df['client'] = ''
# #fill df['client'] with random client ids corresponding to device id
# """for i in range(0, len(devicearr)):
#   for j in range(0, len(df)):
#     if(df['device_id'][j] == devicearr[i]):
#       df['client'][j] = dfcrf['Masked Client ID'][i]"""
# it = 0
# step = int(len(df)/dfcrf['Masked Client ID'].nunique())
# for i in range(0, len(df), step):
#   for j in range(0,step):
#     df['client'][i+j] = clientarr[it]
    
#   it = it + 1

In [13]:
dfcrf.head()


,Time,Masked Client ID,Date of Review,COVID -19,Which Review is this,Insurance,Any Hospitalizations in the last 30 days,Note on Hospitalization (date and reason),How can we help prevent a return to hospital?,Have you fallen in:,...,Does your Caregiver carry out the care plan satisfactory,Detail any issues,Are the cargivers able to perform all the tasks on your care plan,Detail any issues.1,Are there any upcoming schedule changes?,Details of Schedule changes,Are you happy with the services being provided?,If No make notes here,Any Questions or Concerns?,Case Manager Notes
0,2022-01-03 14:59:17,Client088,2022-01-03,By Phone,5 Day,Banner,No,NaN,NaN,No falls,...,Yes,NaN,Yes,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,"ALL IS WELL FCG, SEEKING SUPPORT FOR MED APPTS..."
1,2022-01-04 13:12:24,Client172,2022-01-04,By Phone,90 Day,UHC,No,NaN,NaN,No falls,...,NaN,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,"CHECKING INTO DD SERVICES, HE HAS APPLIED."
2,2022-01-04 13:31:32,Client012,2022-01-04,By Phone,30 Day,UHC,Yes,COTTONWOOD HOSP DISCHARGED 12.31.2021,UNKNOWN,No falls,...,Yes,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,ALL IS WELL
3,2022-01-06 11:56:52,Client022,2022-01-06,By Phone,90 Day,UHC,No,NaN,NaN,No falls,...,Yes,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,NEW FCG IS APPLYING CARMEN HERNANDEZ
4,2022-01-10 11:03:08,Client087,2022-01-10,By Phone,30 Day,UHC,No,NaN,NaN,No falls,...,Yes,NaN,NaN,NaN,No,NaN,Yes,NaN,ALL IS WELL,RAMS AND RAISE WALKER


In [14]:
dfadl.head()

,DeIdentify ID,ProviderID,PayerGroup,Caregiver ID,VisitDuration,ServiceTypeName,VisitDate,ActualTimeIn,ActualTimeOut,txtBreakTimesValue,VisitDate2,ScheduledTimeIn,ScheduledTimeOut,TaskCodeGroupName,HasBeenDone,Reason,Comments,NoSignature,RectangleVoiceCaptured,SignatureDate
0,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Clean Bathroom (30),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
1,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Clean Kitchen (31),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
2,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Dust (32),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
3,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Laundry (36),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
4,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Linen Change (37),N,Performed on another day,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM


In [15]:
dfadl['date'] = pd.to_datetime(dfadl['VisitDate'])
dfadl['date'] = dfadl['date'].dt.date
dfadl['month_year'] = dfadl['date'].map(lambda x: 100*x.year + x.month)

In [16]:
tempdate = pd.to_datetime(dfcrf['Time'])
dfcrf['date'] = tempdate.dt.date
dfcrf['month_year'] = dfcrf['date'].map(lambda x: 100*x.year + x.month)

In [17]:
dfcrf['client'] = dfcrf['Masked Client ID']
dfadl['client'] = dfadl['DeIdentify ID']

In [100]:
mergedcrfadl = pd.merge(dfadl, dfcrf, on = ["client","month_year"])
mergedcrfadl
mergedcrfadl = mergedcrfadl.sample(n=40000)

In [101]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")

In [102]:
#mergedcrfadl['Any_Hospitalizations_in_the_last_30_days']=mergedcrfadl['Any_Hospitalizations_in_the_last_30_days'].eq('Yes').mul(1)
#mergedcrfadl.Any_Hospitalizations_in_the_last_30_days=pd.Series(np.where(mergedcrfadl.Any_Hospitalizations_in_the_last_30_days.values == 'Yes', 1, 0),mergedcrfadl.index)
mergedcrfadl['Any Hospitalizations in the last 30 days'].replace(['No', 'Yes'],
                        [0, 1], inplace=True)
mergedcrfadl['Any Hospitalizations in the last 30 days'].unique()


array([0, 1])

In [103]:
mergedcrfadl=mergedcrfadl.drop(['Date of Review', 'DeIdentify ID', 'Masked Client ID', 'Comments'], axis = 1)
mergedcrfadl

,ProviderID,PayerGroup,Caregiver ID,VisitDuration,ServiceTypeName,VisitDate,ActualTimeIn,ActualTimeOut,txtBreakTimesValue,VisitDate2,...,Detail any issues,Are the cargivers able to perform all the tasks on your care plan,Detail any issues.1,Are there any upcoming schedule changes?,Details of Schedule changes,Are you happy with the services being provided?,If No make notes here,Any Questions or Concerns?,Case Manager Notes,date_y
41001,127318.0,United Community Plan / United Community Plan ...,Caregiver039,1 h,AC Respite,2022-07-13,12:00:00,13:00:00,NaN,2022-07-13,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,ALL IS WELL HAPPY WITH HER CCG.,2022-07-28
34354,127318.0,United Community Plan / United Community Plan ...,Caregiver089,6 h,AC Family non resident,2022-04-15,08:00:00,14:00:00,NaN,2022-04-15,...,NaN,NaN,NaN,No,NaN,Yes,NaN,"ALL IS WELL||MAY 13, 2022 VERY SPECIAL DAY FOR...",FCG WILL BE ASSISTING,2022-04-25
3,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-03-01,10:00:00,13:00:00,NaN,2022-03-01,...,NaN,NaN,NaN,No,NaN,Yes,NaN,ALL IS WELL,IN THE NEAR FUTURE WE WILL NEED A MALE CG FOR ...,2022-03-15
99051,486278.0,DDD / DDD / DDD,Caregiver162,2 h,DDD-Respite,2022-06-20,12:00:00,14:00:00,NaN,2022-06-20,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NONE,Everything is good with client and caregivers ...,2022-06-21
90086,486278.0,DDD / DDD / DDD,Caregiver162,5 h,DDD-Attendant Care,2022-06-06,06:00:00,11:00:00,NaN,2022-06-06,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NONE,"Did video 90 day review , everything is well ...",2022-06-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97620,486278.0,DDD / DDD / DDD,Caregiver162,1 h,DDD-Respite,2022-01-25,14:30:00,15:30:00,NaN,2022-01-25,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NONE,Jesus is happy and feels comfortable with Res...,2022-01-10
68372,486278.0,DDD / DDD / DDD,Caregiver149,12 h,DDD-Respite,2022-05-08,07:00:00,19:00:00,NaN,2022-05-08,...,NaN,NaN,NaN,No,NaN,Yes,NaN,Mom asked in FCG Ana is able to clock out out ...,Client is doing well. Looking forward to summe...,2022-05-18
38726,127318.0,BANNER - UFC / BANNER - UFC / BANNER - UFC,Caregiver099,6 h 30m,AC Respite,2022-08-21,12:30:00,19:00:00,NaN,2022-08-21,...,NaN,NaN,NaN,No,NaN,Yes,NaN,Everything is good with both caregivers and c...,Everything is great with the both caregivers a...,2022-08-30
65124,127318.0,United Community Plan / United Community Plan ...,Caregiver145,3 h,AC Family Resident,2022-02-09,05:00:00,08:00:00,NaN,2022-02-09,...,NaN,NaN,NaN,Yes,"HAD TO GO TO CALIFORNIA/ DAD, EXTREMLY SICK PE...",Yes,NaN,NOT AT THIS TIME,HAS A DR. APPT TOMORROE 2.8.2022,2022-02-07


In [104]:
mergedcrfadl=mergedcrfadl.drop(['ProviderID', 'PayerGroup', 'VisitDuration','VisitDate','txtBreakTimesValue', 'VisitDate2', 'ScheduledTimeIn','Date and Place of last HbA1C Check',
       'Date and place of last Mammogram (if female)',
       'Date and place of last Colorectal Exam',
       'Date and place of last time you had your cholesterol checked',
       'Date and place of last Bone Density Scan',
       'Does your health plan meet your needs?',
       'Has the Care Plan been reviewed with the Client',
       'Any changes to current care plan', 'Notes on changes to Care Plan',
       'Do your caregivers arrive on time?', 'Details of timekeeping issues',
       'Does your Caregiver carry out the care plan satisfactory',
       'Detail any issues',
       'Are the cargivers able to perform all the tasks on your care plan',
       'Detail any issues.1', 'Are there any upcoming schedule changes?',
       'Details of Schedule changes',
       'Are you happy with the services being provided?',
       'If No make notes here', 'Any Questions or Concerns?',
       'Case Manager Notes', 'date_y', 'Notes on changes to medical conditions','Notes on new medical devises', 'Any changes to Mental Status',
       'Notes on changes to mental status', 'How is your vision?',
       'How is your hearing', 'Do you take your medications',
       'Do you fill all your prescriptions?',
       'How would you describe your diet',
       'How much water do you drink in a day?','Note on Hospitalization (date and reason)',
       'How can we help prevent a return to hospital?','Which Review is this',
       'Insurance',
       'Have you fallen in:', 'Did your fall result in',
       'Matrix (Do you worry about falling?)',
       'Matrix (Do you have a Living Will?)', 'Matrix (Do you have a  DNR?)',
       'Do you have a POA', 'Do you need help with DNR, Living Will or POA',
       'Note on help needed?',
       'Are there any changes to your home environment since the last Intake/Review?',
       'Condition of the Home',
       'Are you receiving Home Health Services', 'If so from which Agency',
       'Are you expecting a Nurse or Therapist to visit you at home?',
       'What will the Nurse/Therapist do?',
       'Any changes to Transportation needs?', 'Notes on transportation needs',
       'Have you had any new DME equipment in the last 90 days?',
       'Notes on new DME equipment', 'Do you need any DME Equipment?',
       'Notes on DME Equipment needed',
       'Any new medical devises in the past 90 days', 'How do you sleep',
       'What do you do about your medical appointments','Where and when did you have your flu shot?','Caregiver ID', 'ServiceTypeName', 'ActualTimeIn', 'ActualTimeOut',
       'ScheduledTimeOut', 'Reason','NoSignature', 'RectangleVoiceCaptured', 'SignatureDate', 'date_x'], axis = 1)
mergedcrfadl.columns

Index(['TaskCodeGroupName', 'HasBeenDone', 'month_year', 'client', 'Time',
       'COVID -19', 'Any Hospitalizations in the last 30 days',
       'Has anyone moved in our out in the last 30 days?',
       'Any new medical conditions in the last 90 days',
       'Have you had your flu shot this year?'],
      dtype='object')

In [105]:
#mergedcrfadl=mergedcrfadl.drop(['COVID -19','Have you had your flu shot this year?','Any new medical conditions in the last 90 days'],axis = 1)
mergedcrfadl_data = pd.get_dummies(mergedcrfadl, columns = ['Has anyone moved in our out in the last 30 days?','client','TaskCodeGroupName', 'HasBeenDone','COVID -19','Have you had your flu shot this year?','Any new medical conditions in the last 90 days'])
#mergedcrfadl_data = mergedcrfadl_data.sample(n=40000)

In [106]:
arima_fit = auto_arima(mergedcrfadl_data['Any Hospitalizations in the last 30 days'], trace = True, suppress_warnings=True)
arima_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-16720.068, Time=15.38 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-16725.361, Time=7.71 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-16725.594, Time=3.40 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-16725.578, Time=4.62 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-15088.231, Time=1.65 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=-16724.090, Time=4.18 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-16723.643, Time=5.64 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-16722.110, Time=38.73 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-15175.906, Time=0.97 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 82.301 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                40000
Model:               SARIMAX(1, 0, 0)   Log Likelihood                8365.797
Date:                Sat, 10 Dec 2022   AIC                         -16725.594
Time:                        11:33:46   BIC                         -16699.804
Sample:                             0   HQIC                        -16717.431
                              - 40000                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0399      0.132      0.301      0.764      -0.220       0.299
ar.L1          0.0075      0.005      1.461      0.144      -0.003       0.017
sigma2         0.0385      0.011      3.474      0.001       0.017       0.060
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):            809387.21
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.92   Skew:                             4.68
Prob(H) (two-sided):                  0.00   Kurtosis:                        22.95
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [164]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [165]:
trainingset = mergedcrfadl_data[:-4000]
testingset = mergedcrfadl_data[-4000:]

In [166]:
arima_model = sm.tsa.arima.ARIMA(trainingset['Any Hospitalizations in the last 30 days'],order=(1,0,0))
arima_model = arima_model.fit()
arima_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                          SARIMAX Results                                           
====================================================================================================
Dep. Variable:     Any Hospitalizations in the last 30 days   No. Observations:                36000
Model:                                       ARIMA(1, 0, 0)   Log Likelihood                7462.199
Date:                                      Sat, 10 Dec 2022   AIC                         -14918.398
Time:                                              12:35:27   BIC                         -14892.925
Sample:                                                   0   HQIC                        -14910.295
                                                    - 36000                                         
Covariance Type:                                        opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0403      0.172      0.235      0.814      -0.296       0.377
ar.L1          0.0061      0.005      1.122      0.262      -0.005       0.017
sigma2         0.0387      0.014      2.695      0.007       0.011       0.067
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):            722194.69
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.92   Skew:                             4.67
Prob(H) (two-sided):                  0.00   Kurtosis:                        22.85
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [167]:
start = len(trainingset)
end = len(trainingset)+len(testingset)-1
predctn = arima_model.predict(start=start,end=end,type='levels')
print(predctn)
predctn.unique()

36000    0.040059
36001    0.040304
36002    0.040305
36003    0.040305
36004    0.040305
           ...   
39995    0.040305
39996    0.040305
39997    0.040305
39998    0.040305
39999    0.040305
Name: predicted_mean, Length: 4000, dtype: float64


array([0.04005857, 0.04030352, 0.04030502, 0.04030503, 0.04030503,
       0.04030503, 0.04030503, 0.04030503])

In [172]:
print(trainingset['Any Hospitalizations in the last 30 days'].mean())
ptemp = []
for p in predctn:
  if p > trainingset['Any Hospitalizations in the last 30 days'].mean():
    ptemp.append(1)
  else:
    ptemp.append(0)

predctn = ptemp

0.04030555555555555


In [173]:
 from sklearn.metrics import f1_score

In [174]:
truevals = testingset['Any Hospitalizations in the last 30 days']
f1_score(truevals, predctn, average='macro')

0.49012109623964306